In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --quiet adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.6 MB/s eta 0:00:0000:0100:01


# Imports

In [3]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from art.estimators.classification import PyTorchClassifier
from art.defences.trainer import AdversarialTrainerAWPPyTorch
from art.utils import load_cifar10
from art.attacks.evasion import ProjectedGradientDescent

# Load Data

In [4]:
def prepare_data():
    """Load and prepare CIFAR-10 data."""
    print("Loading CIFAR-10 dataset...")

    # Load CIFAR-10
    (x_train, y_train), (x_test, y_test), _, _ = load_cifar10()

    # Convert to PyTorch format (N, C, H, W)
    x_train = x_train.transpose(0, 3, 1, 2).astype("float32")
    x_test = x_test.transpose(0, 3, 1, 2).astype("float32")

    print(f"Training data shape: {x_train.shape}")
    print(f"Training labels shape: {y_train.shape}")
    print(f"Test data shape: {x_test.shape}")
    print(f"Test labels shape: {y_test.shape}")

    return (x_train, y_train), (x_test, y_test)

In [5]:
(x_train, y_train), (x_test, y_test) = prepare_data()

Loading CIFAR-10 dataset...
Training data shape: (50000, 3, 32, 32)
Training labels shape: (50000, 10)
Test data shape: (10000, 3, 32, 32)
Test labels shape: (10000, 10)


In [6]:
norm_mean = np.array([0.4914, 0.4822, 0.4465]).reshape(3, 1, 1)
norm_std = np.array([0.2471, 0.2435, 0.2616]).reshape(3, 1, 1)

In [7]:
norm_mean = np.tile(norm_mean, (1, 32, 32))
norm_std = np.tile(norm_std, (1, 32, 32))

# Model

In [8]:
class SimpleCNN(nn.Module):
    """
    A simple CNN architecture for CIFAR-10 classification.
    """

    def __init__(self, num_classes: int = 10) -> None:
        super(SimpleCNN, self).__init__()

        self.conv_block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.conv_block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc_layers = nn.Sequential(
            nn.Linear(64 * 8 * 8, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

In [9]:
def init_model_weights(module: nn.Module) -> None:
    """Initialize model weights."""
    if isinstance(module, nn.Conv2d):
        n = module.kernel_size[0] * module.kernel_size[1] * module.out_channels
        module.weight.data.normal_(0, math.sqrt(2.0 / n))
        if module.bias is not None:
            module.bias.data.zero_()

    elif isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)
        if module.bias is not None:
            module.bias.data.zero_()

In [10]:
def create_classifier(model, norm_mean, norm_std, device):
    """Create ART PyTorchClassifier."""
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

    # Create classifier
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(0.0, 1.0),
        preprocessing=(norm_mean, norm_std),
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 32, 32),
        nb_classes=10,
        device_type=device
    )

    return classifier

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


# Train Standard Model

In [12]:
def train_standard_model(x_train, y_train, norm_mean, norm_std, device='cuda'):
    """Train standard model using ART's fit method."""

    # Create model
    model = SimpleCNN(num_classes=10)
    model.apply(init_model_weights)
    model = model.to(device)

    # Create classifier
    classifier = create_classifier(model, norm_mean, norm_std, device)

    # Train using ART's fit method
    print("Training standard model...")
    classifier.fit(x_train, y_train, batch_size=128, nb_epochs=10)

    print("Standard model training completed!")
    return classifier

In [13]:
standard_classifier = train_standard_model(
    x_train, y_train, norm_mean, norm_std, device
)

Training standard model...
Standard model training completed!


# Train Adversarial Model

In [26]:
def train_adversarial_model(x_train, y_train, norm_mean, norm_std, device='cuda'):
    """Train adversarial model using AdversarialTrainerAWPPyTorch."""
    
    # Create model
    model = SimpleCNN(num_classes=10)
    model.apply(init_model_weights)
    model = model.to(device)

    proxy_model = SimpleCNN(num_classes=10)
    proxy_model.apply(init_model_weights)
    proxy_model = proxy_model.to(device)
    
    # Define optimizers and loss - BUNLAR EKLENDİ
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    proxy_optimizer = torch.optim.SGD(proxy_model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss()

    # Define clip values (CIFAR-10 için)
    min_pixel_value = 0.0
    max_pixel_value = 1.0
    
    # Create classifier - PARAMETRELER DÜZELTİLDİ
    classifier = PyTorchClassifier(
        model=model,
        clip_values=(min_pixel_value, max_pixel_value),
        preprocessing=(norm_mean, norm_std),  # Düzeltildi
        loss=criterion,
        optimizer=optimizer,
        input_shape=(3, 32, 32),
        nb_classes=10,
    )

    proxy_classifier = PyTorchClassifier(
        model=proxy_model,
        clip_values=(min_pixel_value, max_pixel_value),
        preprocessing=(norm_mean, norm_std),  # Düzeltildi
        loss=criterion,
        optimizer=proxy_optimizer,
        input_shape=(3, 32, 32),
        nb_classes=10,
    )

    pgd_attack = ProjectedGradientDescent(
        estimator=classifier,
        norm=np.inf,
        eps=8.0/255.0,
        eps_step=2.0/255.0,
        max_iter=10,
        targeted=False,
        num_random_init=1,
        batch_size=128
    )
    
    # Create adversarial trainer
    adv_trainer = AdversarialTrainerAWPPyTorch(
        classifier, 
        proxy_classifier,
        pgd_attack,
        mode="PGD",
        gamma=0.01,  # proxy model learning rate ile aynı olmalı
        beta=6.0,
        warmup=0
    )
    
    # Train with adversarial examples
    print("Training adversarial model...")
    adv_trainer.fit(x_train, y_train, batch_size=128, nb_epochs=10)
    
    print("Adversarial model training completed!")
    return classifier  # proxy_classifier değil, ana classifier döndür

In [27]:
adversarial_classifier = train_adversarial_model(
    x_train, y_train, norm_mean, norm_std, device
)

Training adversarial model...


Adversarial Training AWP with PGD - Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial model training completed!


# Evaluate Models

In [28]:
def evaluate_models(standard_classifier, adversarial_classifier, x_test, y_test):
    """Evaluate both models on clean and adversarial data."""
    
    # Standard model
    standard_preds = standard_classifier.predict(x_test)
    standard_accuracy = np.sum(np.argmax(standard_preds, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print(f"Standard Model Clean Accuracy: {standard_accuracy * 100:.2f}%")
    
    # Adversarial model
    adversarial_preds = adversarial_classifier.predict(x_test)
    adversarial_accuracy = np.sum(np.argmax(adversarial_preds, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print(f"Adversarial Model Clean Accuracy: {adversarial_accuracy * 100:.2f}%")
    
    # Create separate PGD attacks for each model
    print("Generating adversarial examples for standard model...")
    pgd_attack_standard = ProjectedGradientDescent(
        estimator=standard_classifier,
        norm=np.inf,
        eps=8.0/255.0,
        eps_step=2.0/255.0,
        max_iter=10,
        targeted=False,
        num_random_init=1,
        batch_size=128
    )
    
    # Generate adversarial examples for standard model
    x_test_adv_standard = pgd_attack_standard.generate(x_test)
    
    # Evaluate standard model on adversarial examples
    standard_adv_preds = standard_classifier.predict(x_test_adv_standard)
    standard_adv_accuracy = np.sum(np.argmax(standard_adv_preds, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print(f"Standard Model Adversarial Accuracy: {standard_adv_accuracy * 100:.2f}%")
    
    # Create separate PGD attack for adversarial model
    print("Generating adversarial examples for adversarial model...")
    pgd_attack_adversarial = ProjectedGradientDescent(
        estimator=adversarial_classifier,
        norm=np.inf,
        eps=8.0/255.0,
        eps_step=2.0/255.0,
        max_iter=10,
        targeted=False,
        num_random_init=1,
        batch_size=128
    )
    
    # Generate adversarial examples for adversarial model
    x_test_adv_adversarial = pgd_attack_adversarial.generate(x_test)
    
    # Evaluate adversarial model on adversarial examples
    adversarial_adv_preds = adversarial_classifier.predict(x_test_adv_adversarial)
    adversarial_adv_accuracy = np.sum(np.argmax(adversarial_adv_preds, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print(f"Adversarial Model Adversarial Accuracy: {adversarial_adv_accuracy * 100:.2f}%")
    
    return {
        'standard_clean': standard_accuracy * 100,
        'adversarial_clean': adversarial_accuracy * 100,
        'standard_adv': standard_adv_accuracy * 100,
        'adversarial_adv': adversarial_adv_accuracy * 100
    }

In [29]:
results = evaluate_models(standard_classifier, adversarial_classifier, x_test, y_test)

Standard Model Clean Accuracy: 74.63%
Adversarial Model Clean Accuracy: 10.00%
Generating adversarial examples for standard model...


PGD - Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Standard Model Adversarial Accuracy: 11.48%
Generating adversarial examples for adversarial model...


PGD - Batches:   0%|          | 0/79 [00:00<?, ?it/s]

Adversarial Model Adversarial Accuracy: 10.02%


# Results

In [30]:
print("="*50)
print("FINAL SUMMARY")
print("="*50)
print(f"Standard Model:")
print(f"Clean Accuracy:       {results['standard_clean']:.2f}%")
print(f"Adversarial Accuracy: {results['standard_adv']:.2f}%")
print(f"Robustness Drop:      {results['standard_clean'] - results['standard_adv']:.2f}%")

print(f"\nAdversarial Model:")
print(f"Clean Accuracy:       {results['adversarial_clean']:.2f}%")
print(f"Adversarial Accuracy: {results['adversarial_adv']:.2f}%")
print(f"Robustness Drop:      {results['adversarial_clean'] - results['adversarial_adv']:.2f}%")

robustness_improvement = results['adversarial_adv'] - results['standard_adv']
print(f"\nRobustness Improvement: +{robustness_improvement:.2f}%")

clean_performance_drop = results['standard_clean'] - results['adversarial_clean']
print(f"Clean Performance Drop: {clean_performance_drop:.2f}%")

FINAL SUMMARY
Standard Model:
Clean Accuracy:       74.63%
Adversarial Accuracy: 11.48%
Robustness Drop:      63.15%

Adversarial Model:
Clean Accuracy:       10.00%
Adversarial Accuracy: 10.02%
Robustness Drop:      -0.02%

Robustness Improvement: +-1.46%
Clean Performance Drop: 64.63%
